In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import lightgbm as lgb
import xgboost as xgb
%matplotlib inline

In [ ]:
train = pd.read_csv('data/3/open100.csv')
#train = pd.read_csv('data/3/open100.csv',nrows=15000000)

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(train.data_ind.unique())
print(train.data_ind.unique().shape)

In [ ]:
sss = sorted(train.Z.unique())
min(np.array(sss[1:]) - np.array(sss[:-1]))

In [ ]:
for coord in (['X','Y','Z']):
    train.loc[train.data_ind == 15].hist(column=coord, by='signal', bins=150)
    train.loc[train.data_ind == 15].hist(column=coord, bins=150)
plt.show()

In [ ]:
y, x = np.histogram(train[(train.data_ind == 15) & (train.signal == 1)].X, bins=300)
plt.plot(x[:-1], y)

In [ ]:
def trace_start(df, ind_id):
    if 'signal' in df.columns:
        df_ind = df[(df.data_ind == ind_id) & (df.signal == 1)].copy()
    else:
        df_ind = df[df.data_ind == ind_id].copy()
    y, x = np.histogram(df_ind.X, bins=100)
    idY_max = np.argmax(y)
    dfX_max = df_ind[(df_ind.X > x[idY_max - 1]) & (df_ind.X < x[idY_max + 1])]
    return dfX_max[dfX_max.id == dfX_max.X.idxmax()]

In [ ]:
trace_start(train, 15)

In [ ]:
train[(train.data_ind == 15) & (train.signal == 1)].head()

In [ ]:
def update_DataFrame(df):
    new_df = []
    for data_ind in df.data_ind.unique():
        df_ind = df[df.data_ind == data_ind].copy()
        start = trace_start(df, data_ind)
        
        df_ind['X'] = df_ind.X - start.iloc[0].X
        df_ind['Y'] = df_ind.Y - start.iloc[0].Y
        df_ind['Z'] = df_ind.Z - start.iloc[0].Z
        new_df.append(df_ind)
    return pd.concat(new_df)

In [ ]:
test = pd.read_csv('data/3/test_50.csv')

In [ ]:
print(test.shape)
test.head()

In [ ]:
print(sorted(test.data_ind.unique()))

In [ ]:
selInd = 2727

In [ ]:
for coord in (['X','Y','Z']):
    test.loc[test.data_ind == selInd].hist(column=coord, bins=100)
plt.show()

In [ ]:
trace_start(test, selInd)

In [ ]:
trainUpdated = update_DataFrame(train)
trainUpdated.shape

In [ ]:
trainUpdated.head()

In [ ]:
X_train = trainUpdated.drop(columns=['id', 'signal'])
y_train = trainUpdated.signal

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'boost_from_average':'false',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'max_depth': 15,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 4
}

In [ ]:
nRounds = 20
lgb.cv(params, lgb_train, nRounds, nfold=3)

In [ ]:
nRounds = 100
bst = lgb.train(params, lgb_train, nRounds)

In [ ]:
updated_test = update_DataFrame(test)
X_test = updated_test.drop(columns=['id'])

In [ ]:
y_pred = bst.predict(X_test)

In [ ]:
result = pd.DataFrame(data={'signal': y_pred})
result.insert(0, 'id', range(len(result)))

In [ ]:
print(result.shape)
result.head()

In [ ]:
result.to_csv('data/3/submit.csv',index=False)